# M31S 

Closed economy 2x2 with taxes in the benchmark


| Markets | X   | Y   | W   |     | CONS|
| ---     | --- | --- | --- | --- | --- |
| PX      | 100 |     | -100|     |     |
| PY      |     | 100 | -100|     |     |
| PW      |     |     | 200 |     | -200|
| PL      | -20 | -60 |     |     | 80  |
| PK      | -60 | -40 |     |     | 100 |
|         |     |     |     |     |     |
| TAX     | -20 |     |     |     | 20  |

In [16]:
using MPSGE
using DataFrames

In [ ]:
M31 = MPSGEModel()

tx0 = 0
ty0 = 0
tlx0 = 1
tkx0 = 0

@parameters(M31, begin
    TX, tx0,  (description = "Proportional output tax on sector X",)
    TY, ty0,  (description = "Proportional output tax on sector Y",)
    TLX, tlx0, (description = "Ad-valorem tax on labor inputs to X",)
    TKX, tkx0, (description = "Ad-valorem tax on captial inputs to X",)
end)

@sectors(M31, begin
    X
    Y
    W
end)

@commodities(M31, begin
    PX
    PY
    PW
    PL
    PK
end)

@consumer(M31, CONS)

@production(M31, X, [s=1, t=0], begin
    @output(PX, 100, t, taxes = [Tax(CONS, TX)])#, reference_price = 1 - tx0)
    @input(PL, 20, s, taxes = [Tax(CONS, TLX)], reference_price = 1+tlx0)
    @input(PK, 60, s, taxes = [Tax(CONS,TKX)])#, reference_price = 1+tkx0
end)


@production(M31, Y, [s=1, t=0], begin
    @output(PY, 100, t, taxes = [Tax(CONS, TY)])
    @input(PL, 60, s)
    @input(PK, 40, s)
end)

@production(M31, W, [s=1, t=0], begin
    @output(PW, 200, t)
    @input(PX, 100, s)
    @input(PY, 100, s)
end)

@demand(M31, CONS, begin
    @final_demand(PW, 200)
    @endowment(PL, 80)
    @endowment(PK, 100)
end);

# Benchmark Calibration

In [6]:
fix(PW, 1)

solve!(M31; cumulative_iteration_limit=0)
generate_report(M31)

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_208F.tmp
 > cumulative_iteration_limit 0
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris
Preprocessed size   : 8

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     1     1 0.0000e+00           I 0.0e+00 0.0e+00 (zero_profit[X)

Major Iterations. . . . 0
Minor Iterations. . . . 0
Restarts. . . . . . . . 0
Crash Iterations. . . . 0
Gradient Steps. . . . . 0
Function Evaluations. . 1
Gradient Evaluations. . 1
Basis Time. . . . . . . 0.000000
Total Time. . . . . . . 0.000000
Residual. . . . . . . . 0.000000e+00
Postsolved residual: 0.0000e+00


Solver Status: LOCALLY_SOLVED
Model Status: FEASIBLE_POINT

Row,var,value,margin
,GenericV…,Float64,Float64
1,X,1.0,0.0
2,W,1.0,0.0
3,Y,1.0,0.0
4,PL,1.0,0.0
5,PW,1.0,0.0
6,PK,1.0,0.0
7,PY,1.0,0.0
8,PX,1.0,0.0
9,CONS,200.0,0.0


# First Counterfactual

Replace tax on labor inputs by a uniform tax on both factors

In [13]:
set_value!(TLX, .25)
set_value!(TKX, .25)
set_value!(TX, 0)
set_value!(TY, 0)

solve!(M31)
CF1 = generate_report(M31)

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_CA7B.tmp
 > cumulative_iteration_limit 10000
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Preprocessed size   : 8

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             2.9730e+01             0.0e+00 (market_clearance[PK)
    1     1     0     8 1.8637e+00  1.0e+00    0.0e+00 (zero_profit[Y)
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     2     2 1.8637e+00           I 0.0e+00 1.2e+00 (zero_profit[Y)
    1     1     3     3 1.6985e-03  1.0e+00 SO 0.0e+00 1.4e-03 (income_balance[)
    2     1     4     4 4.9963e-09  1.0e+00 SO 0.0e+00 2.8e-09 (market_clearanc)

Major Iterations. . . . 2
Minor Iterations. . . . 2
Restarts. . . . . . . . 0
Crash Iterations. . . . 1
Gradient Steps. . . . . 0
F

Row,var,value,margin
,GenericV…,Float64,Float64
1,X,1.07779,-2.42731e-9
2,W,1.02132,-2.13532e-10
3,Y,0.967806,-2.70312e-9
4,PL,1.17452,1.86778e-9
5,PW,1.0,-9.77184e-9
6,PK,0.898759,2.79859e-9
7,PY,1.05529,5.49392e-11
8,PX,0.947605,1.68782e-10
9,CONS,204.263,6.05382e-10


# Second Counterfactual

Demonstrate that a 25% tax on all inputs is equivalent to a 20% tax on the output (or all outputs if more than one)

In [14]:
set_value!(TLX, 0)
set_value!(TKX, 0)
set_value!(TX, .2)
set_value!(TY, 0)

solve!(M31)
CF2 = generate_report(M31)

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_E19E.tmp
 > cumulative_iteration_limit 10000
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Preprocessed size   : 8

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             2.9103e+01             0.0e+00 (market_clearance[PK)
    1     1     0     8 1.7156e+00  1.0e+00    0.0e+00 (zero_profit[Y)
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     2     2 1.7156e+00           I 0.0e+00 1.2e+00 (zero_profit[Y)
    1     1     3     3 9.8865e-04  1.0e+00 SO 0.0e+00 7.7e-04 (market_clearanc)
    2     1     4     4 4.7414e-09  1.0e+00 SO 0.0e+00 2.8e-09 (market_clearanc)

Major Iterations. . . . 2
Minor Iterations. . . . 2
Restarts. . . . . . . . 0
Crash Iterations. . . . 1
Gradient Steps. . . . . 0
F

Row,var,value,margin
,GenericV…,Float64,Float64
1,X,1.07779,-1.94179e-9
2,W,1.02132,-2.13475e-10
3,Y,0.967806,-2.70313e-9
4,PL,1.17452,1.86775e-9
5,PW,1.0,-9.77604e-9
6,PK,0.898759,2.7986e-9
7,PY,1.05529,5.4996e-11
8,PX,0.947605,1.68797e-10
9,CONS,204.263,7.78471e-11


# Counterfactual 3

Finally, demonstrate that a 20% tax on the $X$ sector output is equivalent to a 25% subsidy on $Y$ sector output (assumes that the funds for the subsidy can be raised lump sum from the consumer!) 

In [15]:
set_value!(TLX, 0)
set_value!(TKX, 0)
set_value!(TX, 0)
set_value!(TY, -.25)

solve!(M31)
CF3 = generate_report(M31)

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_FB80.tmp
 > cumulative_iteration_limit 10000
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Preprocessed size   : 8

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             8.3931e+01             0.0e+00 (zero_profit[Y)
    1     1     0     8 3.1997e+00  1.0e+00    0.0e+00 (market_clearance[PK)
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     2     2 3.1997e+00           I 0.0e+00 2.8e+00 (market_clearanc)
    1     1     3     3 7.4371e-02  1.0e+00 SO 0.0e+00 5.3e-02 (market_clearanc)
    2     1     4     4 3.0502e-05  1.0e+00 SO 0.0e+00 2.3e-05 (market_clearanc)
    3     1     5     5 6.0521e-12  1.0e+00 SO 0.0e+00 4.3e-12 (market_clearanc)

Major Iterations. . . . 3
Minor Iterations. . . 

Row,var,value,margin
,GenericV…,Float64,Float64
1,X,1.07779,-2.43006e-12
2,W,1.02132,-2.27374e-13
3,Y,0.967806,-3.42126e-12
4,PL,1.46814,4.30589e-12
5,PW,1.0,-1.24203e-11
6,PK,1.12345,-3.97904e-13
7,PY,1.05529,4.68958e-13
8,PX,0.947605,-1.98952e-13
9,CONS,204.263,-8.52651e-14


# Verifying Solutions Match

In [19]:
innerjoin(CF1, CF2, on = :var; makeunique=true) |>
    x-> innerjoin(x, CF3, on = :var, makeunique=true) |>
    x -> select(x, :var, :value, :value_1, :value_2)

Row,var,value,value_1,value_2
,GenericV…,Float64,Float64,Float64
1,X,1.07779,1.07779,1.07779
2,W,1.02132,1.02132,1.02132
3,Y,0.967806,0.967806,0.967806
4,PL,1.17452,1.17452,1.46814
5,PW,1.0,1.0,1.0
6,PK,0.898759,0.898759,1.12345
7,PY,1.05529,1.05529,1.05529
8,PX,0.947605,0.947605,0.947605
9,CONS,204.263,204.263,204.263
